# RAG系统实现教程

## 1. 环境变量配置
首先配置OpenAI API的环境变量，包括API密钥和基础URL。

In [1]:
# 导入必要的库
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

# 加载 .env 文件中的环境变量
load_dotenv()

# 从环境变量获取API密钥
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

# 检查环境变量是否正确加载
if not api_key:
    raise ValueError("请设置 OPENAI_API_KEY 环境变量")
if not base_url:
    raise ValueError("请设置 OPENAI_BASE_URL 环境变量")

print("环境变量加载成功！")
print(f"API密钥前缀: {api_key[:10]}...")
print(f"Base URL: {base_url}")

环境变量加载成功！
API密钥前缀: sk-proj-R_...
Base URL: https://ai.devtool.tech/proxy/v1


## 2. 初始化OpenAI客户端
使用环境变量创建OpenAI客户端实例。

In [2]:
# 实例化客户端 - 使用环境变量
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

print("OpenAI 客户端创建成功！")

OpenAI 客户端创建成功！


## 3. 导入RAG系统模块
导入自定义的RAG系统相关类，包括Embedding、文件读取、向量存储和聊天功能。

In [3]:
from RAG import OpenAIEmbedding, ReadFiles, VectorStore, GPT4oChat

## 4. 测试OpenAI Embedding API
直接调用OpenAI的embedding API来测试文本向量化功能。

In [4]:
# 调用 embedding API 获取文本的向量表示
response = client.embeddings.create(
    input="测试文本",  # 输入文本
    model="text-embedding-3-small"  # 选择 Embedding 模型
)

# 打印返回的embedding向量
print(response.data[0].embedding[:10])

# 获取embedding长度
len(response.data[0].embedding)

[0.0060808719135820866, 0.01937379501760006, 0.01978626847267151, -0.04902195185422897, 0.005962129216641188, -0.05264672636985779, -0.026648342609405518, -0.012949194759130478, 0.0021545535419136286, -0.02132367342710495]


1536

## 5. 使用自定义Embedding类
使用封装好的OpenAIEmbedding类来获取文本的向量表示。

In [5]:
# 初始化 Embedding 模型
embedding_model = OpenAIEmbedding()

# 输入需要获取向量表示的文本
text = "这是一个示例文本，用于演示 OpenAI Embedding 的使用。"

# 获取文本的向量表示
embedding_vector = embedding_model.get_embedding(text, model="text-embedding-3-large")

print("文本的向量表示为：", embedding_vector[:10])
print("长度为：", len(embedding_vector))

文本的向量表示为： [0.026480568572878838, 0.013057298958301544, -0.007077604997903109, -0.010436690412461758, 0.019500991329550743, 0.013423269614577293, -0.00191317533608526, 0.035316139459609985, 0.0011117983376607299, -0.026480568572878838]
长度为： 3072


## 6. 计算文本相似度
通过比较两个文本的向量表示来计算它们的余弦相似度。

In [6]:
text1 = '我喜欢吃苹果'
text2 = "苹果是我最喜欢吃的水果"
text3 = "我喜欢用苹果手机"

vector1 = embedding_model.get_embedding(text1)
vector2 = embedding_model.get_embedding(text2)

similarity = OpenAIEmbedding.cosine_similarity(vector1, vector2)
print(f"两段文本的余弦相似度为: {similarity}")

两段文本的余弦相似度为: 0.7661800739633337


## 7. 重新导入RAG模块
为了确保使用最新的模块代码，重新导入RAG系统相关类。

In [7]:
from RAG import OpenAIEmbedding, ReadFiles, VectorStore, GPT4oChat

## 8. 读取文档文件
初始化文件读取器，获取指定目录下的所有支持的文件。

In [8]:
# 初始化 ReadFiles 类，指定文件目录路径
file_reader = ReadFiles(path="./data")

# 获取目录下所有支持的文件类型
file_list = file_reader.get_files()
print("支持的文件列表：", file_list)

支持的文件列表： ['./data/【人工智能】AI构建者手册2025 | ICONIQ发布68页报告| AI原生公司 | AI赋能公司 | 代理工作流 | 基础设施 | 市场定价 | 团队结构 | 成本预算 | 内部效率 - YouTube.txt']


## 9. 文档分块处理
将读取的文档内容按指定长度进行分块，便于后续的向量化处理。

In [9]:
# 将文件内容读取并分块
document_chunks = file_reader.get_content(max_token_len=600, cover_content=150)
print("分块后的文档内容：", document_chunks)

分块后的文档内容： ['`大家好，这里是最佳拍档，我是大飞。不知道大家有没有发现，最近两年AI领域的讨论已经从“要不要做AI”变成了“怎么把AI做好”。从2023年生成式AI爆发开始，到2025年的今天，越来越多的公司不再纠结于AI的可能性，而是埋头在产品开发、团队搭建、成本控制这些实际的问题上。最近，曾经作为管理硅谷众多科技巨头家办的ICONIQ资本，抓住了这个转折点的时机，发布了一份长达68页的报告，拆解了300家正在开发AI产品的软件公司的实战经验。无论是AI原生公司，还是所谓的AI赋能公司，相信它们的踩坑经验、成功做法，都能带给我们很多的启发。今天大飞就来给大家分享一下。\n\n', '问题上。最近，曾经作为管理硅谷众多科技巨头家办的ICONIQ资本，抓住了这个转折点的时机，发布了一份长达68页的报告，拆解了300家正在开发AI产品的软件公司的实战经验。无论是AI原生公司，还是所谓的AI赋能公司，相信它们的踩坑经验、成功做法，都能带给我们很多的启发。今天大飞就来给大家分享一下。\n\n要聊AI产品的开发，首先我们得搞清楚现在市面上的AI公司到底有哪些类型。根据报告的调研，主要分为两类，分别是AI原生公司和AI赋能公司。AI原生公司指的是核心产品或商业模式完全由AI驱动的公司，它们的价值几乎都来自于模型训练、推理和持续的学习，这类公司在调研中占了32%，它们的特点是产品的迭代速度非常快。报告里提到，只有1%的AI原生公司还停留在产品发布前的阶段，而AI赋能公司则有11%卡在这个阶段；更关键的是，47%的AI原生产品已经进入了规模化的阶段，也就是说产品已经验证了市场契合度，正在快速扩大用户群和基础设施。这背后可能的原因是，AI原生公司在团队构成、基础设施和融资模式上更有优势，能够更快的跳过试错阶段，而AI赋能公司往往需要把AI“嫁接”到现有的工作流中，难免就会遇到更多的阻碍。', '段；更关键的是，47%的AI原生产品已经进入了规模化的阶段，也就是说产品已经验证了市场契合度，正在快速扩大用户群和基础设施。这背后可能的原因是，AI原生公司在团队构成、基础设施和融资模式上更有优势，能够更快的跳过试错阶段，而AI赋能公司往往需要把AI“嫁接”到现有的工作流中，难免就会遇到更多的阻碍。AI赋能公司则分为两种，一种是在旗舰产品中嵌入AI功能，比如给现有的CRM系统加一个A

## 10. 查看分块数量
查看文档被分为了多少个块。

In [10]:
len(document_chunks)

28

## 11. 创建向量数据库
创建向量存储实例，对文档进行向量化并持久化保存。

In [11]:
# 创建向量数据库
vector_store = VectorStore(document=document_chunks)

# 使用 OpenAI Embedding 模型对文档进行向量化
embedding_model = OpenAIEmbedding()

# 获取文档向量并存储
vector_store.get_vector(embedding_model)

# 持久化存储到本地
vector_store.persist('storage')

## 12. 查询相关文档
使用向量相似度搜索来找到与用户查询最相关的文档片段。

In [12]:
# 模拟用户查询
query = "AI公司分为哪两种类型？"
result = vector_store.query(query, embedding_model)

print("检索结果：", result)

检索结果： ['问题上。最近，曾经作为管理硅谷众多科技巨头家办的ICONIQ资本，抓住了这个转折点的时机，发布了一份长达68页的报告，拆解了300家正在开发AI产品的软件公司的实战经验。无论是AI原生公司，还是所谓的AI赋能公司，相信它们的踩坑经验、成功做法，都能带给我们很多的启发。今天大飞就来给大家分享一下。\n\n要聊AI产品的开发，首先我们得搞清楚现在市面上的AI公司到底有哪些类型。根据报告的调研，主要分为两类，分别是AI原生公司和AI赋能公司。AI原生公司指的是核心产品或商业模式完全由AI驱动的公司，它们的价值几乎都来自于模型训练、推理和持续的学习，这类公司在调研中占了32%，它们的特点是产品的迭代速度非常快。报告里提到，只有1%的AI原生公司还停留在产品发布前的阶段，而AI赋能公司则有11%卡在这个阶段；更关键的是，47%的AI原生产品已经进入了规模化的阶段，也就是说产品已经验证了市场契合度，正在快速扩大用户群和基础设施。这背后可能的原因是，AI原生公司在团队构成、基础设施和融资模式上更有优势，能够更快的跳过试错阶段，而AI赋能公司往往需要把AI“嫁接”到现有的工作流中，难免就会遇到更多的阻碍。']


## 13. 生成AI回答
使用GPT-4模型结合检索到的相关文档来生成问题的答案。

In [13]:
chat = GPT4oChat(api_key=api_key)
question = 'AI公司分为哪两种类型？'
answer = chat.chat(question, [], result)

## 14. 显示生成的答案
查看AI生成的回答内容。

In [14]:
answer

'参考段落的内容总结如下：\n\n根据ICONIQ资本发布的一份报告，目前市面上的AI公司主要分为两种类型：AI原生公司和AI赋能公司。AI原生公司是其核心产品或商业模式完全由AI驱动，且它们的商业价值来自于模型的训练、推理和持续学习。这类公司展现出较快的产品迭代速度，47%的产品已经进入规模化阶段。相对而言，AI赋能公司则需要将AI技术融入到现有的工作流中，面临更多的挑战和障碍。\n\n有用的回答是：AI公司分为两种类型，分别是AI原生公司和AI赋能公司。AI原生公司以AI为核心驱动力，而AI赋能公司则将AI技术应用于现有流程中。'

## 15. 格式化显示答案
使用Markdown格式美化显示AI生成的答案。

In [15]:
display(Markdown(answer))

参考段落的内容总结如下：

根据ICONIQ资本发布的一份报告，目前市面上的AI公司主要分为两种类型：AI原生公司和AI赋能公司。AI原生公司是其核心产品或商业模式完全由AI驱动，且它们的商业价值来自于模型的训练、推理和持续学习。这类公司展现出较快的产品迭代速度，47%的产品已经进入规模化阶段。相对而言，AI赋能公司则需要将AI技术融入到现有的工作流中，面临更多的挑战和障碍。

有用的回答是：AI公司分为两种类型，分别是AI原生公司和AI赋能公司。AI原生公司以AI为核心驱动力，而AI赋能公司则将AI技术应用于现有流程中。